In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [2]:
input_path = "../content/DATA_CHAMBER_2021/"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -uq "/content/drive/My Drive/DATA_CHAMBER_2021.zip" -d "./"

In [5]:
class ImageFolderWithPaths(datasets.ImageFolder):

    # called by dataloaders
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [6]:
preprocess = {
    'train':
    transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}

image_datasets = {
    'train': 
    ImageFolderWithPaths(input_path + 'train', preprocess['train']),
    'validation': 
    ImageFolderWithPaths(input_path + 'test', preprocess['validation'])
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','validation']}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=16,
                                shuffle=True,
                                num_workers=2, pin_memory=True),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=16,
                                shuffle=True,
                                num_workers=2, pin_memory=True)
}

In [7]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19_bn', pretrained=False)
model.classifier[6] = nn.Linear(4096,3)
# fine tuning

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


In [8]:
import torchvision
from torch.autograd import Variable
import time
from PIL import Image
from torchvision import transforms

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9)

In [10]:
# t0 = time.time()
# print('{} seconds'.format(time.time() - t0)
epochLine = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [11]:
def train_model(model, criterion, optimizer, num_epochs=1):
    train_batches = len(dataloaders['train'])
    lossLine = []
    accLine = []
    timeLine = []
    for epoch in range(num_epochs):
        print('-' * 40)
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 20)
        t0 = time.time()

        for phase in ['train']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            training_loss = 0.0
            training_corrects = 0

            for i,data in enumerate(dataloaders[phase]):
                inputs, labels,_ = data
                print("\rTraining batch {}/{}".format(i+1, train_batches), end='', flush=True)
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                # loss.requires_grad = True

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                training_loss += loss.item() * inputs.size(0)
                training_corrects += torch.sum(preds == labels.data)

            epoch_loss = training_loss / len(image_datasets[phase])
            lossLine.append(epoch_loss)
            epoch_acc = training_corrects.double() / len(image_datasets[phase])
            accLine.append(epoch_acc)

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
            t1 = time.time()
            t = t1 - t0
            print('Duration :{} seconds'.format(t))
            ts = pd.DataFrame([t]).apply(np.float32)
            timeLine.append(ts)
    return model, lossLine, accLine, timeLine

In [12]:
model = model.to(device)

In [13]:
model_trained, lossLine, accLine, timeLine = train_model(model, criterion, optimizer, num_epochs=20)

----------------------------------------
Epoch 1/20
--------------------
Training batch 420/420train loss: 0.5259, acc: 0.7853
Duration :286.97650480270386 seconds
----------------------------------------
Epoch 2/20
--------------------
Training batch 420/420train loss: 0.0683, acc: 0.9748
Duration :286.7766613960266 seconds
----------------------------------------
Epoch 3/20
--------------------
Training batch 420/420train loss: 0.0232, acc: 0.9927
Duration :286.59425497055054 seconds
----------------------------------------
Epoch 4/20
--------------------
Training batch 420/420train loss: 0.0010, acc: 0.9997
Duration :286.77442240715027 seconds
----------------------------------------
Epoch 5/20
--------------------
Training batch 420/420train loss: 0.0002, acc: 1.0000
Duration :286.5803370475769 seconds
----------------------------------------
Epoch 6/20
--------------------
Training batch 420/420train loss: 0.0001, acc: 1.0000
Duration :286.7462692260742 seconds
-------------------

In [14]:
plt.plot(epochLine, lossLine, label="Loss")
plt.plot()

plt.xlabel("Epochs")
plt.ylabel("Loss rate")
plt.title("VGG 19 bn Loss per Epoch")
plt.legend()
plt.show()

In [15]:
print(lossLine)

[0.5258899274328883, 0.0683189411465291, 0.023173931406051732, 0.0010239834254733688, 0.00016938297627709471, 9.771145062746223e-05, 6.472388035209278e-05, 7.378601160721106e-05, 4.884800624723814e-05, 6.727374318918648e-05, 7.911708156031764e-05, 3.808967201895325e-05, 4.441273566370923e-05, 3.2108117698201965e-05, 3.030544862826159e-05, 2.1186453212231514e-05, 2.0539935439598904e-05, 2.275844132500867e-05, 2.375114483804009e-05, 1.3296113172573519e-05]


In [16]:
plt.plot(epochLine, accLine, label="Accuracy")
plt.plot()

plt.xlabel("Epochs")
plt.ylabel("Accuracy rate")
plt.title("VGG 19 bn Accurate per Epoch")
plt.legend()
plt.show()

In [17]:
print(accLine)

[tensor(0.7853, device='cuda:0', dtype=torch.float64), tensor(0.9748, device='cuda:0', dtype=torch.float64), tensor(0.9927, device='cuda:0', dtype=torch.float64), tensor(0.9997, device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype=torch.float64), tensor(1., device='cuda:0', dtype

In [18]:
total_time_train = 0
for i in timeLine:  total_time_train += i
print(total_time_train)

             0
0  5772.975098


In [19]:
def test_model(model, criterion, optimizer):
    labels_input=list()
    labels_output=list()
    vid_id = list()
    for phase in ['validation']:
        model.eval()

        for inputs, labels, fname in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels_input= labels_input + labels.tolist()
            for f in fname:
                vid_id.append(f.split('/')[-1].split('.')[0].split('_')[0])
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            
            labels_output= labels_output + preds.tolist()
    return labels_input,labels_output,vid_id
            
y_true,y_pred,vid_id = test_model(model, criterion, optimizer)

In [20]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(classification_report(y_true,y_pred))
accuracy_score(y_true, y_pred)

              precision    recall  f1-score   support

           0       0.82      0.65      0.72       409
           1       0.43      0.55      0.48       367
           2       0.64      0.62      0.63       831

    accuracy                           0.61      1607
   macro avg       0.63      0.61      0.61      1607
weighted avg       0.63      0.61      0.62      1607



0.611076540136901

In [21]:
df = pd.DataFrame(list(zip(y_true,y_pred,vid_id)),
               columns =['y_true','y_pred','vid_id'])
df.to_csv('df.csv',encoding='utf-8',index=False)

In [22]:
vid_list = list(set(df['vid_id'].values))

y_true = []
y_pred = []
for vid in vid_list:
    #print(vid)
    tmp_df = df[df['vid_id']==vid]
    #print(len(tmp_df))
    vid_pred = tmp_df['y_pred'].mode().values[0]
    vid_label = tmp_df['y_true'].mode().values[0]
    y_true.append(vid_label)
    y_pred.append(vid_pred)
    #print(vid_label,"\n",vid_pred)
    
    print('vid: {} label: {} pred: {}'.format(vid,vid_label,vid_pred))

vid: 193 label: 2 pred: 2
vid: 166 label: 1 pred: 1
vid: 170 label: 2 pred: 2
vid: 192 label: 0 pred: 0
vid: 165 label: 0 pred: 0
vid: 182 label: 2 pred: 1
vid: 185 label: 1 pred: 1
vid: 179 label: 1 pred: 2
vid: 173 label: 2 pred: 2
vid: 180 label: 2 pred: 1
vid: 188 label: 2 pred: 1
vid: 172 label: 2 pred: 2
vid: 158 label: 0 pred: 0
vid: 161 label: 1 pred: 1
vid: 175 label: 1 pred: 1
vid: 176 label: 0 pred: 0
vid: 187 label: 2 pred: 2
vid: 184 label: 2 pred: 1
vid: 169 label: 0 pred: 0
vid: 174 label: 1 pred: 1
vid: 168 label: 0 pred: 0
vid: 178 label: 0 pred: 0
vid: 189 label: 1 pred: 2
vid: 163 label: 2 pred: 2
vid: 190 label: 1 pred: 1
vid: 171 label: 0 pred: 2
vid: 195 label: 2 pred: 2
vid: 186 label: 1 pred: 2
vid: 164 label: 2 pred: 2
vid: 191 label: 0 pred: 0
vid: 194 label: 1 pred: 2
vid: 162 label: 1 pred: 2
vid: 167 label: 2 pred: 2
vid: 157 label: 1 pred: 2
vid: 160 label: 2 pred: 2
vid: 177 label: 0 pred: 0
vid: 183 label: 0 pred: 0
vid: 159 label: 1 pred: 1
vid: 181 lab

In [23]:
accuracy_score(y_true,y_pred)

0.717948717948718